## Kafka Consumer
**Intended Utility**
>This databrick is one piece of a "producer - consumer" pair. <br>
>It is intended to simulate a stream of data between a source and a datalake.<br>

>In this scenario, the source is an Azure Blob, and the data is Zillow Home Price Data.<br>
>The data that is consumed in this consumer will be saved to an Azure Blob, and then the <br>
>the database will be periodically updated from the blob.

**Configuration:**
> These cells are responsible for configuring the primary aspects of the databrick.<br>

**Config Part 1:** Import Libraries

In [0]:
#pip install confluent-kafka
#shouldn't need to do this

**Error Callbacks**

In [0]:
def error_cb(err):
    """ The error callback is used for generic client errors. These
        errors are generally to be considered informational as the client will
        automatically try to recover from all errors, and no extra action
        is typically required by the application.
        For this example however, we terminate the application if the client
        is unable to connect to any broker (_ALL_BROKERS_DOWN) and on
        authentication errors (_AUTHENTICATION). """

    print("Client error: {}".format(err))
    if err.code() == KafkaError._ALL_BROKERS_DOWN or \
       err.code() == KafkaError._AUTHENTICATION:
        # Any exception raised from this callback will be re-raised from the
        # triggering flush() or poll() call.
        raise KafkaException(err)


### Helpful Links

* [Confluent's github repo](https://github.com/confluentinc/confluent-kafka-python) - Confluent's github repo of code examples for python Kafka examples, includes almost everything needed for core development with Kafka
* [Docstring Documentation](https://www.datacamp.com/community/tutorials/docstrings-python) - Comments on Page made in Docstring

**Kafka Consumer Setup**

In [0]:
from confluent_kafka import Consumer
from time import sleep
import uuid
from confluent_kafka import Producer, Consumer, KafkaError, KafkaException
import json


#KAFKA variables, Move to the OS variables or configuration
# This will work in local Jupiter Notebook, but in a databrick, hiding config.py is tougher. 
confluentClusterName = "stage3talent"
confluentBootstrapServers = "pkc-ldvmy.centralus.azure.confluent.cloud:9092"
#confluentTopicName = "cell-jed"
confluentTopicName = "arctic_analysts_main_table"
schemaRegistryUrl = "https://psrc-gq7pv.westus2.azure.confluent.cloud"
confluentApiKey = "YHMHG7E54LJA55XZ"
confluentSecret = "/XYn+w3gHGMqpe9l0TWvA9FznMYNln2STI+dytyPqtZ9QktH0TbGXUqepEsJ/nR0"
confluentRegistryApiKey = "YHMHG7E54LJA55XZ"
confluentRegistrySecret = "/XYn+w3gHGMqpe9l0TWvA9FznMYNln2STI+dytyPqtZ9QktH0TbGXUqepEsJ/nR0"


#Kakfa Class Setup.
c = Consumer({
    'bootstrap.servers': confluentBootstrapServers,
    'sasl.mechanism': 'PLAIN',
    'security.protocol': 'SASL_SSL',
    'sasl.username': confluentApiKey,
    'sasl.password': confluentSecret,# this will create a new consumer group on each invocation.
    'group.id': str(1),
    'auto.offset.reset': 'earliest',
    'enable.auto.commit': True,
    'error_cb': error_cb,
})

#c.subscribe(['cell-jed'])
c.subscribe(['arctic_analysts_main_table'])


**Define the received string storage mode and a list to contain received items**

In [0]:
# Function to check for messages until 1000, and then commiting message

def check_for_messages():
    aString = {}
    kafkaListDictionaries = []

    i = 0
    while True:
        if len(kafkaListDictionaries) > 1000: # Bumped up for error checking
            result = 'update'
            break
        try:
            # Pause and wait for new messages
            msg = c.poll(timeout=15)

            # If timeout and no new messages
            if msg is None:
                i += 1
                plural = "." if 10 - i == 1 else "s."
                print(f'{15*i} seconds elapsed without a response.')
                if i == 10:
                    result = 'timeout'
                    break

                print(f'Stopping Consumer after {10 - i} more attempt{plural}')

            elif msg.error():
                print("Consumer error: {}".format(msg.error()))
                break
            else:
                i = 0
                print(f'Message {len(kafkaListDictionaries)} successfully received')
                aString=json.loads('{}'.format(msg.value().decode('utf-8')))
#                 aString['timestamp'] = msg.timestamp()[1]
                print(aString)
                kafkaListDictionaries.append(aString)

                c.commit(asynchronous=False)
        except Exception as e:
            print(e)
            break
    print('Consumer Stopped')
    return kafkaListDictionaries, result

In [0]:
# Function to read from database table to check to only write new data to database table

def add_to_database(dataframe):
    database = "arctic_analysts_capstone"
    table = "dbo.main_table"
    user = "arctic_analysts"
    password  = "ThisPassw0rd!"
    server = "gen10-data-fundamentals-22-02-sql-server.database.windows.net"

    
    read_from_df = spark.read.format("jdbc") \
      .option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
      .option("dbtable", table) \
      .option("user", user) \
      .option("password", password) \
      .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
      .load()
    
    write_df = dataframe.join(read_from_df, ['FIPS','YearID','MonthID'],'left_anti')

    # WRITE <--- dataframe to database
    write_df.write.format("jdbc") \
      .option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
      .mode("append") \
      .option("dbtable", table) \
      .option("user", user) \
      .option("password", password) \
      .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
      .save()
    print('Successfully added the data to the dataframe.')

    
#     READ <--- FROM DATABASE
#     jdbc = spark.read.format("jdbc") \
#       .option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
#       .option("dbtable", table) \
#       .option("user", user) \
#       .option("password", password) \
#       .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
#       .load()

In [0]:
# Calling on functions to check for new messages and writing to database

result = 'started'
while result != 'timeout':
    
    kafkaListDictionaries, result = check_for_messages()
    print(result)
    
    if len(kafkaListDictionaries) > 0:
        try:
#             df = convert_to_dataframe(kafkaListDictionaries)
            df = spark.createDataFrame(kafkaListDictionaries)
            add_to_database(df)
        except Exception as E:
            print(E)
            print('Process Failure')
            break
    else:
        print('There is no new data.')
        
print('Finished Consuming')
# FYI, I have built a pause into the producer at i % 100 == 0, so that is why it occasionally doesn't find any messages.